<a href="https://colab.research.google.com/github/anshkaggarwal22/VirtualPowerPlantProject/blob/main/Fit_Hourly_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Read BEopt Files and fit with Demand/Usage


In [ ]:
# libraries used to deal with dates in the plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import datetime as dt
import os

In [ ]:
from google.colab import drive
WORK_DRIVE = '/content/drive'
drive.mount(WORK_DRIVE)

Mounted at /content/drive


In [ ]:
# Creating folder/file paths
WORK_AREA = WORK_DRIVE + '/MyDrive/Ansh Aggarwal/Models/Unit Commitment and Economic Dispatch'
os.chdir(WORK_AREA)

beopt_path = './BEopt/'
inputs_path = './inputs/'
outputs_path = './outputs/'
data_filepath = beopt_path + 'Normalize_BEopt_and_Demand.csv'

Find Daily Total Energy Based on 5 BEopt Sites

In [ ]:
df_daily_total_energy = pd.read_csv(beopt_path + 'Hourly_Total_E_kWH_5_Sites_73_F.csv')

df_daily_total_energy.info()
df_daily_total_energy.head()

In [ ]:
df_daily_total_energy['Hours'] = df_daily_total_energy['Whole Hours since 00:00 Jan 1']

df_daily_total_energy['Date'] = pd.to_datetime(df_daily_total_energy['Hours since 00:00 Jan 1'], unit='h', origin=pd.Timestamp('01/01/2019')).dt.floor('D')

daily_energy = df_daily_total_energy.groupby(['Date', 'Hours']).agg({'Total (E)': 'sum'})

daily_energy = df_daily_total_energy.groupby('Date')['Total (E)'].sum()

df_india_data = {'Date': [], 'Total Energy(kWH)':[]}

for date, kw in daily_energy.items():
    df_india_data['Date'].append(date)
    df_india_data['Total Energy(kWH)'].append(kw)
    print(f'{date} : Total Energy(kWH): {kw}')

#df_india_data.to_csv(beopt_path + 'placeholder.csv', index=False)

Find Daily Demand for Northern Region

In [ ]:
df_daily_demand = pd.read_csv(data_filepath)

df_daily_demand.info()
df_daily_demand.head()

In [ ]:
mask_r = df_daily_demand.Regions.isin(['NR'])
df_r = df_daily_demand[mask_r].copy()

df_r['Daily_Demand_Date'] = df_r['Daily_Demand_Date'].str.split(' ').str[0]
df_r['Daily_Demand_Date'] = pd.to_datetime(df_r['Daily_Demand_Date'], format = '%d/%m/%Y', errors='coerce')

df_daily_usage = df_r.groupby('Daily_Demand_Date')['Usage'].sum()

df_india_demand = {'Daily_Demand_Date': [], 'Usage(MW)':[]}

for date, usage in df_daily_usage.items():
    df_india_demand['Daily_Demand_Date'].append(date)
    df_india_demand['Usage(MW)'].append(usage)

#df_india_demand.to_csv(beopt_path + 'placeholder.csv', index=False)

Code to Find Normalization Factors and Hourly Demand



In [ ]:
df_hourly_demand = pd.read_csv(data_filepath)

df_hourly_demand.info()
df_hourly_demand.head()

In [ ]:
#NORMALIZATION FACTORS

# Create a new list to store the normalization factors
normalization_factors = []

# Calculate the normalization factors for each set of 24 rows
for i in range(len(df_hourly_demand)):
  day_energy = df_hourly_demand['Total Energy(kWh) 70F'][i // 24]  #Reads which index(day) energy needs to be read based on i(hours) value
  norm_factor = df_hourly_demand['Total E 70F (kWh)'][i] /day_energy #Divides hourly by total day energy
  normalization_factors.append(norm_factor)   #Adds to list


df_daily_normalization = pd.DataFrame({'Daily_Energy_Date': df_hourly_demand['Daily_Energy_Date'],'Normalization Factor': normalization_factors})

print(df_daily_normalization)

#df_daily_normalization.to_csv(beopt_path + 'placeholder.csv', index=False)

#-------------------------------------------------------------------------------------------------------------------------------------------

#HOURLY DEMAND

# Creates a new list for the hourly demand
hourly_demand = []

# Calculate the demand factors for each set of 24 hours
for i in range(len(df_hourly_demand)):
  day_demand = df_hourly_demand['Usage(MW)'][i//24]  #Reads which index(Day) demand needs to be read based on i(hours) value
  demand = df_hourly_demand['Normalization Factor 70F'][i]*day_demand   #Multiplies normalization factor by day demand to find/fit hourly demand
  hourly_demand.append(demand)   #Adds to list

df_demand = pd.DataFrame({'Daily_Energy_Date': df_hourly_demand['Daily_Energy_Date'],'Demand': hourly_demand})

print(df_demand)

#df_demand.to_csv(beopt_path + 'placeholder.csv', index=False)


Create graphs for the Hourly Demand



In [ ]:
df_hourly_demand = pd.read_csv(data_filepath)

df_hourly_demand.info()
df_hourly_demand.head()

In [ ]:
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(8)

plt.title('Northern India: Hourly Demand May 30', size=18)
plt.xlabel('DateTime', size =14)
plt.ylabel('Demand (GW)', size =14)

df_hourly_demand['Hourly_Demand_Date'] = pd.to_datetime(df_hourly_demand['Hourly_Demand_Date'])

day = df_hourly_demand[df_hourly_demand['Hourly_Demand_Date'].dt.strftime('%m/%d/%Y') == '05/30/2019']

# Format plot with 1 hour intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))

plt.plot(day['Hourly_Demand_Date'], day['Demand_76_F_MW'].values, label='Cooling Point: 76F', color='green')
plt.plot(day['Hourly_Demand_Date'], day['Demand_73_F_MW'].values, label='Cooling Point: 73F', color='blue')
plt.plot(day['Hourly_Demand_Date'], day['Demand_70_F_MW'].values, label='Cooling Point: 70F', color='red')
plt.gcf().autofmt_xdate() #GCF means 'get the current figure'

plt.legend()

# Makes graph from 0:00 to 23:00
plt.xlim(pd.Timestamp('05/30/2019 0:00'), pd.Timestamp('05/30/2019 23:00'))

plt.savefig(outputs_path + '/Hourly Demands/' + 'May30_Hourly_Demands.png')
plt.show()